In [167]:
import json
import seaborn as sns
import pandas as  pd
import matplotlib.pyplot as plt
import pickle
import itertools
import numpy as np

In [3]:
with open('users_df.pickle', 'rb') as handle:
    users_df = pickle.load(handle)

In [77]:
with open('elite_years_df.pickle', 'rb') as handle:
    elite_years_df = pickle.load(handle)

In [333]:
top_elite_users_df = elite_years_df.groupby('user_id').size().reset_index()
top_elite_users_df.columns = ['user_id', 'counts']
top_elite_users_df = top_elite_users_df.nlargest(15, 'counts')
top_elite_users_df.head(2)

,user_id,counts
684,-eVlHbM2KRZQnBqfTXtDvw,13
2922,23J4vG9_xxxdnmi8CBX7Ng,13


In [334]:
with open('review_df.pickle', 'rb') as handle:
    review_df = pickle.load(handle)
len(review_df)

5261669

In [130]:
elite_user_review_df = pd.DataFrame(pd.merge(review_df, top_elite_users_df, on='user_id'))

In [335]:
#elite_user_review_df.head(5)
len(elite_user_review_df)

623

In [336]:
elite_user_review5_df = pd.DataFrame(elite_user_review_df[elite_user_review_df['stars'] == 5])
len(elite_user_review5_df)

149

In [338]:
elite_user_review5_df.head(2)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,counts
0,VbOwWJKLC6k3urXIBWIDJA,16,2007-09-22,7,CE1YPdhhZxQ2yJ_UApprcg,5,1742,8,23J4vG9_xxxdnmi8CBX7Ng,13
2,ikubvyZFO0kxhA56RETzIg,0,2004-12-24,1,mNNTXbRPA6xKsWAFEJekdA,5,283,1,23J4vG9_xxxdnmi8CBX7Ng,13


In [339]:
year = elite_user_review5_df['date'].apply(lambda row: pd.to_datetime(row).year)
business_year_df = pd.DataFrame()
business_year_df['business_id'] = elite_user_review5_df['business_id']
business_year_df['year'] = year
business_year_df.head(2)

,business_id,year
0,VbOwWJKLC6k3urXIBWIDJA,2007
2,ikubvyZFO0kxhA56RETzIg,2004


In [340]:
br_df = pd.DataFrame(pd.merge(business_year_df, review_df, on=['business_id']))
year_prv = br_df['date'].apply(lambda row: pd.to_datetime(row).year - 1)
year_nxt = br_df['date'].apply(lambda row: pd.to_datetime(row).year + 1)
br_df['year_p'] = year_prv
br_df['year_n'] = year_nxt
br_df.head(2)

,business_id,year,cool,date,funny,review_id,stars,text,useful,user_id,year_p,year_n
0,VbOwWJKLC6k3urXIBWIDJA,2007,16,2007-09-22,7,CE1YPdhhZxQ2yJ_UApprcg,5,1742,8,23J4vG9_xxxdnmi8CBX7Ng,2006,2008
1,VbOwWJKLC6k3urXIBWIDJA,2007,4,2011-01-27,3,T5taDRkLIr2lA_A2IvU1IA,5,732,4,KbVt_z98dP7O1Khkw56AhA,2010,2012


In [341]:
br_yp_df = pd.DataFrame(br_df[br_df['year'] == br_df['year_p']])
br_yn_df = pd.DataFrame(br_df[br_df['year'] == br_df['year_n']])

In [342]:
print(len(br_yp_df))
print(len(br_yn_df))
br_yn_df.head(1)

7819
8170


,business_id,year,cool,date,funny,review_id,stars,text,useful,user_id,year_p,year_n
812,uGupeWqih0yIcCg8anM1PA,2008,0,2007-12-10,1,AvtpSzKL3yjWB4uy3mzYQQ,2,941,0,zD1T8JFx3O1Rf9aNu49rpQ,2006,2008


In [343]:
br_yp_df = br_yp_df.groupby('business_id').agg({'stars' : np.mean}).reset_index()
br_yp_df = br_yp_df[br_yp_df['stars'] <= 3.5]

br_yn_df = br_yn_df.groupby('business_id').agg({'stars' : np.mean}).reset_index()

In [344]:
print(len(br_yp_df))
print(len(br_yn_df))
br_yp_df.head(2)

20
91


,business_id,stars
9,4zqoILJLZ-3F8VVNr2hg1w,3.0
15,79V8osIXJft6FIixcnFRww,1.0


In [345]:
br_yn_df.head(2)

,business_id,stars
0,-F-hHAOj55_KGCh4-ZfnCg,3.800000
1,-sAr-LA9TsIdl37UjwBlvg,4.163043


In [346]:
pd.DataFrame(pd.merge(br_yp_df, br_yn_df, on='business_id'))

,business_id,stars_x,stars_y
0,79V8osIXJft6FIixcnFRww,1.000000,3.636364
1,GJ_bXUPv672YwNg4TneJog,2.883495,3.166667
2,L-zHt7gZITG1z8l7TZ4dUg,3.500000,1.000000
3,R6jpwPBe9edy0P64uyaLKA,3.353535,2.768421
4,WUq8HJHIZU4uteB154XN7w,3.374126,3.695988
5,ahG8XQK5Aj4kMtOrC8zfTA,3.333333,4.250000
6,dc3uoAmNo5STqKV6mlD_aA,3.250000,4.000000
7,lKom12WnYEjH5FFemK3M1Q,3.200000,5.000000
8,makqSKZaz1aUipoWBW0BsA,3.066667,2.888889
9,n9JeKS94hJRvnyZLqUbO6w,3.125000,3.600000
